In [ ]:
import os
from dotenv import load_dotenv
import csv
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from IPython.display import display
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn.functional as F
import matplotlib.pyplot as plt
import openai
import base64
import io

In [ ]:
load_dotenv()

DATA_DIR = "data"

# Hyperparameters
NUM_FRAMES = 5  # Number of frames to extract from the video
TIME_BETWEEN_FRAMES = 1  # Time between frames in seconds

In [ ]:
train_csv_path = f"{DATA_DIR}/train.csv"
train_videos_folder = f"{DATA_DIR}/train/"

df = pd.read_csv(train_csv_path)
# df = df[df['target']==1]
df.head()

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, data, videos_folder, transform=None):
        self.data = data
        self.videos_folder = videos_folder
        self.transform = transform  # Any image transformations (e.g., augmentations)
        self.num_frames = NUM_FRAMES
        self.time_between_frames = TIME_BETWEEN_FRAMES

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        vid_path = os.path.join(
            self.videos_folder, f"{str(int(row['id'])).zfill(5)}.mp4"
        )

        time = 0 if np.isnan(row["time_of_event"]) else row["time_of_event"]
        images = self.get_multiple_frames(vid_path, time)

        label = row["target"]

        # Apply transformations
        if self.transform:
            images = [self.transform(image) for image in images]

        return images, torch.tensor(label), row["id"]  # Convert label to tensor

    def get_frame(self, video_path, time):
        cap = cv2.VideoCapture(video_path)
        cap.set(cv2.CAP_PROP_POS_MSEC, time * 1000)

        success, frame = cap.read()
        cap.release()

        if not success:
            raise ValueError(f"Failed to read frame at {time} seconds.")

        # Convert BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return Image.fromarray(frame)

    def get_multiple_frames(self, video_path, time):
        time_between_frames = self.time_between_frames

        if time == 0:
            # Get the first frame
            frame = self.get_frame(video_path, time)
            return [frame]

        if time is None:
            # Get the last frames
            cap = cv2.VideoCapture(video_path)
            cap.set(cv2.CAP_PROP_POS_AVI_RATIO, 1)
            time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
            time_between_frames = time / self.num_frames
            cap.release()

        frames = []
        for i in range(self.num_frames):
            try:
                frame = self.get_frame(video_path, time - i * time_between_frames)
                frames.append(frame)
            except ValueError:
                break
        return frames


transform = transforms.Compose(
    [
        transforms.Resize((480, 854)),  # Resize to a standard size
    ]
)

dataset = ImageDataset(df, train_videos_folder, transform=transform)
dataloader = DataLoader(dataset, shuffle=True)

In [ ]:
images, label, id = dataset[np.random.randint(0, len(dataset))]

num_images = len(images)
fig, axes = plt.subplots(1, num_images, figsize=(24, 3))

if num_images == 1:
    axes.imshow(images[0])
    axes.axis("off")
else:
    for i, image in enumerate(images):
        axes[i].imshow(image)
        axes[i].axis("off")

plt.suptitle(f"Label: {int(label)}, ID: {int(id)}")
plt.show()

In [ ]:
def get_llm_prompt(label):
    instruction = f"""
    You are an expert in accident reconstruction and traffic analysis. You will analyze a sequence of dashcam images with a chain of thought reasoning to determine whether there is an immediate threat of vehicle collision. In this particular scenario, there was {"a collision" if label else "no collision"}. Consider each of the following factors:

    1. Vehicle Positions: Identify the locations of all vehicles in each frame and how they change over time.
    2. Trajectories: Determine the direction, speed, and acceleration of each vehicle by comparing their positions across frames.
    3. Nearby Vehicles and Traffic: Identify surrounding vehicles, pedestrians, and any traffic congestion that could impact movement.
    4. Traffic Signals: Consider whether traffic signals indicate a stop, go, or caution state and how that affects the vehicle interactions. Pay special attention on whether vehicles are vialating or obeying traffic signal rules.
    5. Road Conditions and Visibility: Note any obstructions, road markings, or weather conditions that could contribute to the situation.
    """

    return instruction


def encode_image(image):
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def convert_to_conversation(images, label):
    # Combine all images into a single conversation
    conversation = [
        {
            "role": "system",
            "content": [
                {"type": "text", "text": get_llm_prompt(int(label))},
            ],
        },
        {
            "role": "user",
            "content": [
                *[
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        },
                    }
                    for img in images
                ],
            ],
        },
    ]
    return conversation

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")


def generate_response(conversation):
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=conversation,
        n=1,
    )
    return response.choices[0].message.content.strip()

In [ ]:
responses = []

with open(f"gpt4o_cot.csv", mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["id", "label", "response"])
    writer.writeheader()

    for i in tqdm(range(len(dataset))):
        images, label, id = dataset[i]
        conversation = convert_to_conversation(images, label)
        response = generate_response(conversation)
        response_data = {"id": int(id), "label": int(label), "response": response}
        responses.append(response_data)
        writer.writerow(response_data)

responses_df = pd.DataFrame(responses)
responses_df.head()